In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
credits.head(1)

In [ ]:
movies = movies.merge(credits, on = 'title')  # merging both dataframes on the basis of title

In [ ]:
movies.shape

In [ ]:
movies.head(1)

In [ ]:
# important features -> genres, id, keywords, title, overview, cast, crew
movies = movies[['id', 'title', 'overview', 'keywords', 'genres', 'cast', 'crew']]

In [ ]:
movies.head()

## Creating Tags

In [ ]:
movies.isnull().sum()

In [ ]:
# 3 null values in overview column is a very small number so we'll drop them
movies.dropna(inplace=True)
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
def convert2(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 3:
            L.append(i['name'])
            count +=1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert2)

In [ ]:
movies.head()

In [ ]:
def fetch_director(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
# overview is a column of string, so we'll covert it into list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
# now we'll remove space
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(' ', '') for i in x])

In [ ]:
movies.head()

In [ ]:
# now dataset is ready to make tags

movies['tags'] = movies['overview'] + movies['keywords'] + movies['genres'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
# now ther is no need of overview, keywords, genre, cast, crew

tags_df = movies[['id', 'title', 'tags']]

In [ ]:
tags_df

In [ ]:
tags_df['tags'] = tags_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
tags_df['tags'] = tags_df['tags'].apply(lambda x: x.lower())

In [ ]:
tags_df

In [ ]:
# now we'll do text vetorization
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [ ]:
vectors = cv.fit_transform(tags_df['tags']).toarray()

In [ ]:
vectors.shape

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors).shape

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
sorted(list(enumerate(similarity[0])), reverse=True, key = lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = tags_df[tags_df['title'] == movie].index[0]
    distance = similarity[movie_index]
    movie_r = sorted(list(enumerate(distance)), reverse=True, key = lambda x:x[1])[1:6]
    for i in movie_r:
        print(tags_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Batman')

In [ ]:
recommend('John Carter')